In [ ]:
pip install djitellopy opencv-python numpy

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time

#####################################
# Initialization of coordinates
#####################################
# x points straight, y points left, z points up
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
BOX_HALF = BOX_SIZE / 2  # 100 units
box_x_min = ego_x - BOX_HALF
box_x_max = ego_x + BOX_HALF
box_y_min = ego_y - BOX_HALF
box_y_max = ego_y + BOX_HALF

#####################################
# Pursuer position (stationary)
#####################################
# Pursuer position (will be set after takeoff to same z-height)
pursuer_x = 100  # Fixed x position of pursuer
pursuer_y = 50   # Fixed y position of pursuer
pursuer_z = None  # Will be set to takeoff height after takeoff

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
radius = 50
pursuer_threshold = radius**2  # 50^2 = 2500 (50 cm threshold)

#####################################
# Initialization of motion params
#####################################
upward_initialization = 30
left_movement  = 50
right_movement = 50
forward_movement = 50
backward_movement = 50

#####################################
# Initialization of drone
#####################################
tello = Tello()
print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def cbf_filter(ego_x, ego_y, pursuer_x, pursuer_y,
               desired_dx, desired_dy, alpha=0.1):

    # Compute safety margin
    h = (ego_x - pursuer_x)**2 + (ego_y - pursuer_y)**2 - radius**2

    # Approximate derivative of h (treat desired_d* as velocity)
    h_dot = 2*(ego_x - pursuer_x)*desired_dx + 2*(ego_y - pursuer_y)*desired_dy

    # Check condition
    if h_dot >= -alpha * h:
        return desired_dx, desired_dy

    #so i guess now we can't move freely, so
    dx = pursuer_x - ego_x
    dy = pursuer_y - ego_y
    dist = (dx**2 + dy**2)**0.5

    if dx * desired_dx + dy * desired_dy > 0:
        desired_dx = 0
        desired_dy = 0
        print("CBF active: pause movement")

    # Return possibly adjusted movement
    return desired_dx, desired_dy


#####################################
# Drone maneuverance after takeoff
#####################################
try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            pursuer_z = ego_z  # Set pursuer z to same height as takeoff height
            state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
            print(f"Drone moved up 30 cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
            print(f"Pursuer set at position: ({pursuer_x}, {pursuer_y}, {pursuer_z})")
        
        if drone_in_air:
            # W key - Move forward
            if key == ord('w'):
                desired_dx, desired_dy = forward_movement, 0
                safe_dx, safe_dy = cbf_filter(ego_x, ego_y, pursuer_x, pursuer_y, desired_dx, desired_dy)

                # Only move if safe_dx is nonzero
                if safe_dx != 0:
                    tello.move_forward(abs(safe_dx))
                    ego_x += abs(safe_dx)
                    state_history.append((ego_x, ego_y, ego_z))
                    print(f"Moved forward to ({ego_x}, {ego_y}, {ego_z})")
                else:
                    print("CBF filter active")

            # A key - Move left
            elif key == ord('a'):
                desired_dx, desired_dy = 0, left_movement
                safe_dx, safe_dy = cbf_filter(ego_x, ego_y, pursuer_x, pursuer_y, desired_dx, desired_dy)

                if safe_dy != 0:
                    tello.move_left(abs(safe_dy))
                    ego_y += abs(safe_dy)
                    state_history.append((ego_x, ego_y, ego_z))
                    print(f"Moved left to ({ego_x}, {ego_y}, {ego_z})")
                else:
                    print("CBF filter active")

            # S key - Move backward
            elif key == ord('s'):
                desired_dx, desired_dy = -backward_movement, 0
                safe_dx, safe_dy = cbf_filter(ego_x, ego_y, pursuer_x, pursuer_y, desired_dx, desired_dy)

                if safe_dx != 0:
                    tello.move_back(abs(safe_dx))
                    ego_x -= abs(safe_dx)
                    state_history.append((ego_x, ego_y, ego_z))
                    print(f"Moved backward to ({ego_x}, {ego_y}, {ego_z})")
                else:
                    print("CBF filter active")

            # D key - Move right
            elif key == ord('d'):
                desired_dx, desired_dy = 0, -right_movement
                safe_dx, safe_dy = cbf_filter(ego_x, ego_y, pursuer_x, pursuer_y, desired_dx, desired_dy)

                if safe_dy != 0:
                    tello.move_right(abs(safe_dy))
                    ego_y -= abs(safe_dy)
                    state_history.append((ego_x, ego_y, ego_z))
                    print(f"Moved right to ({ego_x}, {ego_y}, {ego_z})")
                else:
                    print("CBF filter active")
                
        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break


#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()
